# LLM as a judge for Summarization Engine - version 2.1

Some improvements to tackle over 2.0:
- Give better understanding of common symptoms and emphasize its inclusion in symptoms list. Specifically orthopnea (requiring pillows during sleep) is often missed in symptoms list.
- Better grasp of sections in a summary JSON output.
- Clarify which types of outputs do not count as diagnosis and are considered okay (should not penalize in `no_diagnosis` output).

In [69]:
import json
from pprint import pprint

import pandas as pd
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from dotenv import load_dotenv

# Load API keys from .env file
load_dotenv()

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 300)

In [56]:
# Create the ChatOpenAI model instance

# model_name = "gpt-3.5-turbo-0125"  # release date: 2024-01-25
# model_name = "gpt-3.5-turbo-1106"  # release date: 2023-11-06
model_name = "gpt-4o-2024-05-13"  # release date: 2024-05-13
# model_name = "gpt-4o-mini"  # release date: 2024-07-18

model = ChatOpenAI(temperature=0.0, model_name=model_name)

### Specify patient transcript file to read in
transcripts_version = 1.0

## System Prompt

In [57]:
# Define the system message for the evaluation

presence_description = "Does the SUMMARY json output contain a section with the key '{topic}'? Note that this criteria looks for the presence of a section, and not whether the section agrees or matches with TRANSCRIPT."

judge_criteria = {
    # summary content
    "intro_patient_present": presence_description.format(topic="patient_overview") + " Also, does the SUMMARY introduce patient by name?",
    "current_symptoms_present": presence_description.format(topic="current_symptoms"),
    "vital_signs_present": presence_description.format(topic="vital_signs"),
    "medications_present": presence_description.format(topic="current_medications"),
    "summary_overview_present": presence_description.format(topic="summary") + " Also, does this section give an overview of the content of the TRANSCRIPT",

    # accuracy of summary
    "symptoms_agree": "Does 'current_symptoms' section in the SUMMARY loosely agree with TRANSCRIPT? Specifically, in SUMMARY, please look in the 'current_symptoms' section only and ignore the 'patient_overview' and 'summary' sections. If there's a symptom that the patient is actively experiencing in 'summary' or 'patient_overview', that symptom must be present in the 'current_symptoms' too. This includes synonym phrases (e.g. 'prop up with pillows' equals to 'orthopnea'). Please look for a loose match only, and ignore the accuracy of mentioned symptoms, i.e. please ignore judging on descriptions of symptom's frequency/severity/details (e.g. 'on and off', 'severe', 'only when moving'). E.g., if TRANSCRIPT says 'chest pain on and off' and SUMMARY says 'chest pain' only, the criteria is still met (output as 1).",
    "vital_signs_agree": "Do the vital signs in the SUMMARY and TRANSCRIPT agree?",
    "meds_agree": "Do the medications in the SUMMARY and TRANSCRIPT agree?",

    # quality of summary
    "no_diagnose": "The SUMMARY is free from interpretation of results (avoided words like 'stable') and is free from diagnosis. Note that narration of patient's words is allowed (like 'patient thinks that they have...' or 'patient is experiencing...' or 'patient confirms that...'); still output a result of 1. Notes of advice like reminder to take meds or monitor symptoms are also allowed. However, predictions of future events (patient is likely/unlikely to...) are interpretations and are not allowed (mark as 0).",
}

system_message_summary_judge = """You are evaluating a summarization engine that has generated a SUMMARY of a doctor-patient dialogue TRANSCRIPT based on a set of criteria. Your evaluation will consist of answering specific questions about the SUMMARY with 1 (Yes) and 0 (No) responses. The SUMMARY quality will depend on the TRANSCRIPT.
{output_format}

CRITERIA (CSV column names, then a description):
""" + "\n".join([f"`{k}`: {v}" for k, v in judge_criteria.items()]) + """

ADDITIONAL INFORMATION: the following are common heart failure symptoms and their descriptions. Any mention of these medical terms or similar phrases do not count as a "diagnosis" in the context of this evaluation. If the patient claims some of these phrases for themselves (e.g. "I need to prop myself up with pillows"), it is a symptom and not a diagnosis, and the symptom should have been included in the symptoms list (e.g. "orthopnea" or "needs pillows" should be present).
- Dyspnea: shortness of breath, whether occurring at rest, walking, or climbing stairs
- Paroxysmal Nocturnal Dyspnea (PND): sudden shortness of breath that wakes patient up at night
- Orthopnea: needing to prop up with pillows to breathe comfortably while lying down
- Edema: swelling in your ankles or legs
- Nocturnal Cough: a cough especially at night
- Chest Pain
- Fatigue and Mental Status: feeling more tired than usual ("feeling tired" and "fatigue" are synonyms), or experience sudden changes in mental clarity (or mental status)

ADDITIONAL INFORMATION: "Do you need to prop yourself up with pillows to breathe comfortably" is a direct question on orthopnea (which is a term that patients don't understand), and as such, "Orthopnea" and "needing to prop up with pillows" are considered as one and the same.
- Orthopnea is often omitted in 'current_symptoms' by the summarizer when a patient claims they need pillows to prop up while laying flat; when this happens, it is a criteria violation for `current_symptoms_agree`.
- However, if the 'current_symptoms' section mentions 'orthopnea' but the TRANSCRIPT says only needing pillows (no explicit mention of 'orthopnea'), it is not a criteria violation (keep the score as 1).

"""

output_csv_format = """Generate a CSV row with the appropriate 1 or 0 for each criteria in the order specified below."""

output_reasoning_format = """In separate lines, state each criteria's value (1 or 0) and briefly explain your reasoning if it's a 0. When explaining reasoning, be very specific and please refer to texts in SUMMARY that is the offender. If it's a 1 (yes), leave the reasoning empty.
Lastly, in one last new line, please provide any short additional observations or suggestions for improvement (2 sentences), but do not repeat evaluation points previously made. Be specific with examples, and be concise with words.
For example:
intro_patient,1,""
current_symptoms,0,"No symptoms are reported in SUMMARY"
vital_signs_agree,0,"Heart rate in SUMMARY is 130, but in TRANSCRIPT it's 131"
meds_agree,0,"Vitamins reported in SUMMARY is not in TRANSCRIPT"
write your one-sentence observation/improvement here
"""

human_message_summary_judge = """
TRANSCRIPT: {transcript}

SUMMARY: {summary}
"""

pprint(system_message_summary_judge.format(output_format=output_reasoning_format), width=120)

('You are evaluating a summarization engine that has generated a SUMMARY of a doctor-patient dialogue TRANSCRIPT based '
 'on a set of criteria. Your evaluation will consist of answering specific questions about the SUMMARY with 1 (Yes) '
 'and 0 (No) responses. The SUMMARY quality will depend on the TRANSCRIPT.\n'
 "In separate lines, state each criteria's value (1 or 0) and briefly explain your reasoning if it's a 0. When "
 "explaining reasoning, be very specific and please refer to texts in SUMMARY that is the offender. If it's a 1 (yes), "
 'leave the reasoning empty.\n'
 'Lastly, in one last new line, please provide any short additional observations or suggestions for improvement (2 '
 'sentences), but do not repeat evaluation points previously made. Be specific with examples, and be concise with '
 'words.\n'
 'For example:\n'
 'intro_patient,1,""\n'
 'current_symptoms,0,"No symptoms are reported in SUMMARY"\n'
 'vital_signs_agree,0,"Heart rate in SUMMARY is 130, but in TRANSCRI

## Import Transcript & Summary Files

In [67]:
# Specify the path to your JSON file
transcripts_json_file_path = f"../../data/patients/patients_{transcripts_version}_with_transcripts_terminated_manual_correct.json"

# Open and read the JSON file
with open(transcripts_json_file_path, 'r') as json_file:
    transcripts = json.load(json_file)

# Specify the path to your summaries JSON file
summaries_json_file_path = f"../../data/patients/patients_{transcripts_version}_summaries.json"

# Open and read the JSON file
with open(summaries_json_file_path, 'r') as json_file:
    summaries = json.load(json_file)

# Specify the CSV file path (make sure it is a file, not a directory)
csv_file_path = f"../../data/evaluations/summaries_{transcripts_version}_evaluation_2.1.csv"

# Additionally make tweaks to JSON keys for this evaluation
keys_to_convert = {
    "Patient Overview": "patient_overview",
    "Current Symptoms": "current_symptoms",
    "Vital Signs": "vital_signs",
    "Current Medications": "current_medications",
    "Summary": "summary",
}
for patient_id, summary in summaries.items():
    new_summary = {}
    for key, value in summary["summary"].items():
        if key in keys_to_convert:
            new_summary[keys_to_convert[key]] = value
        else:
            new_summary[key] = value
    summaries[patient_id]["summary"] = new_summary

# # -----
# # for testing only; comment when not needed -- only try to generate one summary

# # to get a specific patient's transcript
# specific_patient_id = 18136989
# sample_patient_idx = list(transcripts.keys()).index(str(specific_patient_id))

# ## to get a random patient's transcript
# # sample_patient_idx = 0

# transcripts = {k: v for k, v in transcripts.items() if k == list(transcripts.keys())[sample_patient_idx]}
# summaries = {k: v for k, v in summaries.items() if k == list(transcripts.keys())[0]}
# sole_patient_id = list(summaries.keys())[0]

# # tinker -- corrupt the summary to test the evaluator
# summaries[sole_patient_id]["summary"]["Current Medications"] = "Furosemide and Spironolactone"
# summaries[sole_patient_id]["summary"]["Current Symptoms"] += "\nNose bleeding"
# summaries[sole_patient_id]["summary"]["Vital Signs"] = 'Temperature: 98.0 degrees\nHeart Rate: 63 beats per minute\nRespiratory Rate: 23 breaths per minute\nOxygen Saturation: 94.0%\nBlood Pressure: 123/56\nWeight: N/A'
# summaries[sole_patient_id]["summary"]["Summary"] += " Patient is unlikely to relapse."
# # -----

In [68]:
def parse_response(response_content: str, expected_fields=len(judge_criteria)):
    """Function to validate and parse the response.

    Example response:
        'intro_patient,1,""\n'
        'current_symptoms,1,""\n'
        'symptoms_agree,0,"Nose bleeding was mentioned in the summary, but not in the transcript."\n'

    Desired output
        {"intro_patient": {"value": 1, "reasoning": ""}, "current_symptoms": {"value": 1, "reasoning": ""}, ...}
    """
    response_list = response_content.split("\n")[0:expected_fields]
    if len(response_list) != expected_fields:
        return {"error": "Invalid response count"}
    response_dict = {}
    for response in response_list:
        if response:
            # split by first two commas, but keep the rest of the string
            response_split = response.split(",", 2)
            response_dict[response_split[0]] = {"value": int(response_split[1]), "reasoning": response_split[2].strip('"')}

    # remainder text is the observations
    response_dict["observations"] = "\n".join(response_content.split("\n")[expected_fields:]).strip()

    return response_dict

In [70]:
# Write the header to dataframe
# for each criteria, add two columns: one for the value (same name) and one for the reasoning (suffix _reasoning)
column_order = ["transcript_number"]
for criteria in judge_criteria.keys():
    column_order.append(criteria)
    column_order.append(f"{criteria}_reasoning")
column_order.append("observations")

all_rows_series: list[pd.Series] = []
all_responses = []
# Loop through each transcript number, invoke the model, and write the results
for patient_number in transcripts.keys():
    if patient_number in transcripts:
        transcript = transcripts[patient_number]['chat_transcript']
        summary = summaries[patient_number]['summary']

        prompt = (
            SystemMessage(content=system_message_summary_judge.format(output_format=output_reasoning_format))
            + human_message_summary_judge
        )

        # Get the response
        response = model.invoke(prompt.format_messages(transcript=transcript, summary=summary))
        all_responses.append(response)
        response_dict = parse_response(response.content)

        # add to dataframe
        row_to_add = {
            "transcript_number": patient_number,
            **{k: v["value"] for k, v in response_dict.items() if k in judge_criteria.keys()},
            **{f"{k}_reasoning": v["reasoning"] for k, v in response_dict.items() if k in judge_criteria.keys()},
            "observations": response_dict["observations"]
        }
        all_rows_series.append(pd.Series(row_to_add))

# create dataframe
df = pd.DataFrame(all_rows_series)[column_order]

pd.set_option('display.max_columns', None)

display(df)

# Write the dataframe to a CSV file
df.to_csv(csv_file_path, index=False)
print(f"CSV file has been created at: {csv_file_path}")

,transcript_number,intro_patient_present,intro_patient_present_reasoning,current_symptoms_present,current_symptoms_present_reasoning,vital_signs_present,vital_signs_present_reasoning,medications_present,medications_present_reasoning,summary_overview_present,summary_overview_present_reasoning,symptoms_agree,symptoms_agree_reasoning,vital_signs_agree,vital_signs_agree_reasoning,meds_agree,meds_agree_reasoning,no_diagnose,no_diagnose_reasoning,observations
0,12305811,1,,1,,1,,1,,1,,0,"'Fatigue' and 'Edema' are mentioned, but 'shortness of breath with exertion' is missing in 'current_symptoms'.",1,,1,,1,,The summary could be improved by including 'shortness of breath with exertion' in the 'current_symptoms' section to fully capture the patient's reported symptoms.
1,14185111,1,,1,,1,,1,,1,,0,'Fatigue' is listed in 'current_symptoms' but not mentioned in the TRANSCRIPT.,1,,1,,0,'Vital signs are within normal limits' is an interpretation.,The SUMMARY should avoid interpretations like 'Vital signs are within normal limits' and ensure all symptoms listed in 'current_symptoms' are mentioned in the TRANSCRIPT.
2,10339317,1,,1,,1,,1,,1,,0,"'current_symptoms' section in SUMMARY only mentions 'Chest pain' but TRANSCRIPT also mentions 'nocturnal cough, orthopnea, fatigue, and sudden changes in mental status' which should be included as negative symptoms.",0,SUMMARY mentions 'Blood Pressure: N/A' and 'Weight: 180 pounds' which are not present in TRANSCRIPT.,1,,0,"SUMMARY states 'Overall, his symptoms are well managed,' which is an interpretation.","The SUMMARY should avoid adding information not present in the TRANSCRIPT, such as 'Blood Pressure: N/A' and 'Weight: 180 pounds'. Additionally, it should refrain from making interpretative statements like 'Overall, his symptoms are well managed'."
3,14807966,1,,1,,1,,1,,1,,1,,1,,1,,1,,"The summary is comprehensive and captures all the key points from the transcript. However, it could be improved by including more details about the patient's emotional state and the impact of symptoms on daily activities."
4,13912736,1,,1,,1,,1,,1,,0,Lightheadedness upon standing quickly is mentioned in the TRANSCRIPT but not included in the 'current_symptoms' section of the SUMMARY.,1,,1,,1,,"The SUMMARY is generally well-structured and comprehensive. However, it should include all reported symptoms, such as lightheadedness upon standing quickly, in the 'current_symptoms' section for completeness."
5,15338322,1,,1,,1,,1,,1,,0,'Occasional clear phlegm' is mentioned in 'current_symptoms' but not in the TRANSCRIPT as a symptom.,1,,1,,1,,"The SUMMARY could be improved by ensuring that only symptoms explicitly mentioned by the patient are included. For example, 'occasional clear phlegm' should not be listed as a symptom if it was not described as such by the patient."
6,13166275,1,,1,,1,,1,,1,,0,'Needing to prop up with pillows' is mentioned in TRANSCRIPT but not in 'current_symptoms' section of SUMMARY.,0,"Heart Rate in SUMMARY is 122, but in TRANSCRIPT it's 125. Respiratory Rate in SUMMARY is 24, but in TRANSCRIPT it's 26. Oxygen Saturation is missing in SUMMARY but present in TRANSCRIPT as 94.0 percent.",1,,0,"SUMMARY states 'Vital signs show a stable temperature, elevated heart rate, slightly elevated blood pressure, and normal respiratory rate,' which is an interpretation.","The SUMMARY should ensure all symptoms mentioned in the TRANSCRIPT are included in the 'current_symptoms' section. Additionally, vital signs should be accurately transcribed, and interpretations should be avoided."
7,18136989,1,,1,,1,,1,,1,,0,'current_symptoms' section only mentions 'Dyspnea' but should also include 'orthopnea' or 'needing to prop up with pillows'.,1,,1,,0,The phrase 'Vital signs are stable' in the 'summary' section is an interpretation.,"The SUMMARY should include 'orthopnea' or 'needing to prop up with pillows' in the 'current_symptoms' section. Additionally, avoid using interpretative phrases like 'Vital signs are stable' t

CSV file has been created at: ../../data/evaluations/summaries_1.0_evaluation_2.1.csv


In [65]:
parse_response(response.content)

{'intro_patient_present': {'value': 1, 'reasoning': ''},
 'current_symptoms_present': {'value': 1, 'reasoning': ''},
 'vital_signs_present': {'value': 1, 'reasoning': ''},
 'medications_present': {'value': 1, 'reasoning': ''},
 'summary_overview_present': {'value': 1, 'reasoning': ''},
 'symptoms_agree': {'value': 0,
  'reasoning': "'current_symptoms' section only mentions 'Dyspnea' but should also include 'orthopnea' or 'needing to prop up with pillows'."},
 'vital_signs_agree': {'value': 1, 'reasoning': ''},
 'meds_agree': {'value': 1, 'reasoning': ''},
 'no_diagnose': {'value': 0,
  'reasoning': "The phrase 'Vital signs are stable' in the 'summary' section is an interpretation."},
 'observations': "The SUMMARY should include 'orthopnea' or 'needing to prop up with pillows' in the 'current_symptoms' section. Additionally, avoid using interpretative phrases like 'Vital signs are stable' to maintain objectivity."}

In [66]:
# what's an average response resource usage?
response.response_metadata

{'token_usage': {'completion_tokens': 169,
  'prompt_tokens': 2626,
  'total_tokens': 2795},
 'model_name': 'gpt-4o-2024-05-13',
 'system_fingerprint': 'fp_bc2a86f5f5',
 'finish_reason': 'stop',
 'logprobs': None}

In [72]:
patient_id = "13727871"
pprint(transcripts[patient_id]["chat_transcript"], width=120)
pprint(summaries[patient_id]["summary"], width=120)

["Doctor: Hello Anthony Roberts, I'm here to check on how you're feeling today. Let's go over how you've been doing "
 'since your discharge.',
 "Patient: Um... I've been feeling okay, just still dealing with this cough and feeling a bit weak.",
 "Doctor: Thank you for sharing, Anthony. Let's address those concerns. \n"
 '\n'
 'Have you experienced any shortness of breath recently, especially when walking or climbing stairs?',
 "Patient: Um... I haven't noticed any shortness of breath when I walk around the house. It's mostly when I try to do "
 'chores or go outside.',
 'Doctor: Thank you for sharing, Anthony. \n'
 '\n'
 'Have you experienced sudden shortness of breath that wakes you up at night?',
 "Patient: No, I haven't had any sudden shortness of breath that wakes me up at night. Just mostly during the day when "
 "I'm up and about.",
 'Doctor: Do you need to prop yourself up with pillows to breathe comfortably while lying down?',
 'Patient: Yes, sometimes I need to prop myself up

In [73]:
# viewer: open the csv file as a pandas dataframe
df = pd.read_csv(csv_file_path)
display(df)

,transcript_number,intro_patient_present,intro_patient_present_reasoning,current_symptoms_present,current_symptoms_present_reasoning,vital_signs_present,vital_signs_present_reasoning,medications_present,medications_present_reasoning,summary_overview_present,summary_overview_present_reasoning,symptoms_agree,symptoms_agree_reasoning,vital_signs_agree,vital_signs_agree_reasoning,meds_agree,meds_agree_reasoning,no_diagnose,no_diagnose_reasoning,observations
0,12305811,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,0,"'Fatigue' and 'Edema' are mentioned, but 'shortness of breath with exertion' is missing in 'current_symptoms'.",1,NaN,1,NaN,1,NaN,The summary could be improved by including 'shortness of breath with exertion' in the 'current_symptoms' section to fully capture the patient's reported symptoms.
1,14185111,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,0,'Fatigue' is listed in 'current_symptoms' but not mentioned in the TRANSCRIPT.,1,NaN,1,NaN,0,'Vital signs are within normal limits' is an interpretation.,The SUMMARY should avoid interpretations like 'Vital signs are within normal limits' and ensure all symptoms listed in 'current_symptoms' are mentioned in the TRANSCRIPT.
2,10339317,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,0,"'current_symptoms' section in SUMMARY only mentions 'Chest pain' but TRANSCRIPT also mentions 'nocturnal cough, orthopnea, fatigue, and sudden changes in mental status' which should be included as negative symptoms.",0,SUMMARY mentions 'Blood Pressure: N/A' and 'Weight: 180 pounds' which are not present in TRANSCRIPT.,1,NaN,0,"SUMMARY states 'Overall, his symptoms are well managed,' which is an interpretation.","The SUMMARY should avoid adding information not present in the TRANSCRIPT, such as 'Blood Pressure: N/A' and 'Weight: 180 pounds'. Additionally, it should refrain from making interpretative statements like 'Overall, his symptoms are well managed'."
3,14807966,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,"The summary is comprehensive and captures all the key points from the transcript. However, it could be improved by including more details about the patient's emotional state and the impact of symptoms on daily activities."
4,13912736,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,0,Lightheadedness upon standing quickly is mentioned in the TRANSCRIPT but not included in the 'current_symptoms' section of the SUMMARY.,1,NaN,1,NaN,1,NaN,"The SUMMARY is generally well-structured and comprehensive. However, it should include all reported symptoms, such as lightheadedness upon standing quickly, in the 'current_symptoms' section for completeness."
5,15338322,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,0,'Occasional clear phlegm' is mentioned in 'current_symptoms' but not in the TRANSCRIPT as a symptom.,1,NaN,1,NaN,1,NaN,"The SUMMARY could be improved by ensuring that only symptoms explicitly mentioned by the patient are included. For example, 'occasional clear phlegm' should not be listed as a symptom if it was not described as such by the patient."
6,13166275,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,0,'Needing to prop up with pillows' is mentioned in TRANSCRIPT but not in 'current_symptoms' section of SUMMARY.,0,"Heart Rate in SUMMARY is 122, but in TRANSCRIPT it's 125. Respiratory Rate in SUMMARY is 24, but in TRANSCRIPT it's 26. Oxygen Saturation is missing in SUMMARY but present in TRANSCRIPT as 94.0 percent.",1,NaN,0,"SUMMARY states 'Vital signs show a stable temperature, elevated heart rate, slightly elevated blood pressure, and normal respiratory rate,' which is an interpretation.","The SUMMARY should ensure all symptoms mentioned in the TRANSCRIPT are included in the 'current_symptoms' section. Additionally, vital signs should be accurately transcribed, and interpretations should be avoided."
7,18136989,1,NaN,1,NaN,1,NaN,1,NaN,1,NaN,0,'current_symptoms' section only mentions 'Dyspnea' but should also include 'orthopnea' or 'needing to prop up with pillows'.,1,NaN,1,NaN,0,The phrase 'Vital signs are stable' in the 'summary' section is an interpretation.,"The SUMMARY sh

## Idea: prompt improvement

Can we take learnings from this round of evals to improve the original prompt?

In [94]:
[col_reasoning for col_reasoning in [" ".join([str(v) for v in _series if not pd.isna(v)]) for k, _series in df.items() if k.endswith("_reasoning")] if col_reasoning] + list(df["observations"])

["'Fatigue' and 'Edema' are mentioned, but 'shortness of breath with exertion' is missing in 'current_symptoms'. 'Fatigue' is listed in 'current_symptoms' but not mentioned in the TRANSCRIPT. 'current_symptoms' section in SUMMARY only mentions 'Chest pain' but TRANSCRIPT also mentions 'nocturnal cough, orthopnea, fatigue, and sudden changes in mental status' which should be included as negative symptoms. Lightheadedness upon standing quickly is mentioned in the TRANSCRIPT but not included in the 'current_symptoms' section of the SUMMARY. 'Occasional clear phlegm' is mentioned in 'current_symptoms' but not in the TRANSCRIPT as a symptom. 'Needing to prop up with pillows' is mentioned in TRANSCRIPT but not in 'current_symptoms' section of SUMMARY. 'current_symptoms' section only mentions 'Dyspnea' but should also include 'orthopnea' or 'needing to prop up with pillows'. SUMMARY mentions 'left knee pain' but does not include 'no new symptoms related to heart failure' which is a key point 

In [96]:
original_summarization_engine_prompt = """You are a medical assistant tasked with reviewing a transcript of a conversation between a patient and their doctor. You will be provided a transcript. The doctor has asked you to write up a summary of the transcript in the format outlined below. Return your summary as a Python dictionary as follows: {{"patient_overview": "", "current_symptoms": "", "vital_signs": "", "current_medications": "", "summary: ""}}. Ensure the output is in proper dictionary format. The value for each key is a string which contains the text of the summary, including new line characters where appropriate. Add context to symptoms where appropriate, but be brief. List specific medications by name under the appropriate medication category. Do not add any information that is not present in the transcript.

"patient_overview":
    Write a one sentence summary like "[Patient Name] is experiencing [primary symptom or chief complaint]"

"current_symptoms" (Note: Separate each symptom with a new line. Determine if the patient is experiencing any of the following: Dyspnea, Paroxysmal Nocturnal Dyspnea (PND), Orthopnea, Edema, Nocturnal Cough, Chest Pain, Fatigue, Sudden Change in Mental Status.):
    List the symptoms the patient is currently experiencing

"vital_signs" (Note: Separate each vital sign with a new line. Put N/A if not reported in transcript):
    Temperature:
    Heart Rate:
    Respiratory Rate:
    Oxygen Saturation:
    Blood Pressure:
    Weight:

"current_medications" (Note: separate each medication with a new line):
    List the medications the patient is taking.

"summary":
    At a high level, summarize a few key points from the transcript. Include the symptoms that the patient confirms, and the symptoms that the patient denies. Do not list vital sign details in this section.
"""

# individual learnings: df["observations"]
# original prompt: summarization_engine_prompt

improvement_prompt_text = f"""You are tasked with improving a summarization engine's prompt, which generates summaries of doctor-patient dialogues. You will be given a list of learnings generated from an automated evaluation of the engine's summaries. Your task is to provide a revised prompt that addresses the learnings. Return your revised prompt as a string.

ORIGINAL PROMPT:
```
{original_summarization_engine_prompt}
```
"""
improvement_prompt = SystemMessage(content=improvement_prompt_text)

learnings_message = HumanMessage(
    content=(
        "LEARNINGS:\n" + "\n".join(
            [
                col_reasoning for col_reasoning in [
                    " ".join([str(v) for v in _series if not pd.isna(v)])
                    for k, _series in df.items() if k.endswith("_reasoning")
                ]
                if col_reasoning
            ]
            + list(df["observations"])
        )
    )
)

additional_instructions = HumanMessage("""Additionally, apply JSON best practices to keep the outputs processable by downstream systems. After generating the new prompt, please summarize the key changes you made to the prompt under a "KEY CHANGES" section in the response.""")

# Get the response
response = model.invoke((improvement_prompt + learnings_message + additional_instructions).format_messages())

In [97]:
print(response.content)

```json
{
  "prompt": "You are a medical assistant tasked with reviewing a transcript of a conversation between a patient and their doctor. You will be provided a transcript. The doctor has asked you to write up a summary of the transcript in the format outlined below. Return your summary as a JSON object as follows: {\"patient_overview\": \"\", \"current_symptoms\": \"\", \"vital_signs\": \"\", \"current_medications\": \"\", \"summary\": \"\"}. Ensure the output is in proper JSON format. The value for each key is a string which contains the text of the summary, including new line characters where appropriate. Add context to symptoms where appropriate, but be brief. List specific medications by name under the appropriate medication category. Do not add any information that is not present in the transcript.

\"patient_overview\":
    Write a one sentence summary like \"[Patient Name] is experiencing [primary symptom or chief complaint]\"

\"current_symptoms\" (Note: Separate each sympto

In [98]:
response.response_metadata

{'token_usage': {'completion_tokens': 566,
  'prompt_tokens': 2335,
  'total_tokens': 2901},
 'model_name': 'gpt-4o-2024-05-13',
 'system_fingerprint': 'fp_4e2b2da518',
 'finish_reason': 'stop',
 'logprobs': None}